####  사용자 정의 데이터셋
- Pytorch에서 딥러닝 시 대량의 데이터 사용에 따른 부하(H/W, S/W) 및 많은 시간 소요에 대한 해결책으로 제시
- 대량 데이터셋 전용 처리 모듈 제공
- Dataset과 DataLoader
    * DataSet => 데이텃 전처리, 텐서화 등의 작업 진행 
    * DataLoader => Dataset 인스턴스를 사용해서 배치크기 만큼 데이터를 추출함

[1] 모듈 로딩 및 데이터 준비 <hr>

In [132]:
# 모듈로딩
import torch                  
import torch.nn as nn                 
from torch.utils.data import Dataset, DataLoader

import pandas as pd 
from sklearn.preprocessing import LabelEncoder              # 타겟 컬럼 수치화 

In [133]:
# 데이터 
DATA_FILE = '../data/iris.csv'

In [134]:
# CSV ==> DataFrame
irisDF=pd.read_csv(DATA_FILE)
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa


In [135]:
# 타겟 컬럼 수치화 ===> LabelEncoder 
encoder = LabelEncoder()
encoder.fit(irisDF['variety'])
irisDF['variety']=encoder.transform(irisDF['variety'])

In [136]:
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0


[2] 사용자 정의 데이터셋 클래스 생성 <hr>

In [137]:
# -----------------------------------------------------------------
# 클래스목적 : 학습용 데이터셋 텐서화 및 전처리
# 클래스이름 : CustomDataSet
# 부모클래스 : torch.utils.data.Dataset
# 매개  변수 : featureDF, tagetDF
# -----------------------------------------------------------------

In [138]:
class CustomDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self, featureDF, targetDF):
        super().__init__()
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]
    
    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows
    
    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor 반환!!!
    def __getitem__(self, idx):
        featureTS=torch.FloatTensor( self.featureDF.iloc[idx].values )
        targetTS=torch.FloatTensor( self.targetDF.iloc[idx].values )
        return featureTS, targetTS

In [139]:
# -------------------------------------------------------------
# 함수기능 : 파일 확장자별 데이터프레임 변환 기능 
# 함수이름 : convertDataFrame
# 매개변수 : data_path
# 함수결과 : DataFrame
# -------------------------------------------------------------
def convertDataFrame(data_path, exit_header=0):
    ext = data_path.rsplit('.')[-1]
    if ext == 'csv':
        return pd.read_csv(data_path, header=exit_header)
    elif ext == 'json':
        return pd.read_json(data_path, header=exit_header)
    elif ext in ['xlsx', 'xls']:
        return pd.read_excel(data_path, header=exit_header)
    else: 
        return pd.read_table(data_path, header=exit_header)

In [140]:
DATA_PATH1='../data/class02.csv'
dataDF = convertDataFrame(DATA_PATH1)
print( DATA_PATH1, dataDF.head(1) , sep='\n', end='\n\n')

DATA_PATH2='../data/iris.csv'
dataDF = convertDataFrame(DATA_PATH2)
print( DATA_PATH2, dataDF.head(1) , sep='\n')

../data/class02.csv
   5.2  9.8
0  3.0  2.1

../data/iris.csv
   sepal.length  sepal.width  petal.length  petal.width variety
0           5.1          3.5           1.4          0.2  Setosa


In [141]:
### 타겟 DF의 값이 숫자가 아닌경우 인코딩 진행 
def check_type(targetDF):
    if not targetDF.iloc[0].str.isnumeric().item():        
        labels=targetDF[targetDF.columns[0]].unique().tolist()
        label_index=dict(zip(labels, range(len(labels))))        
        return targetDF[targetDF.columns[0]].replace(label_index).copy() 
    else:
        return None


In [142]:
# -------------------------------------------------------------
# 클래스기능 : 파일기반 데이터셋
# 클래스이름 : FileDataset
# 매개  변수 : data_path  파일 경로 
# 부모클래스 : utils.data.Dataset
# -------------------------------------------------------------
class FileDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self, data_path):
        super().__init__()
        # 데이터파일 ==> DataFrame 변환
        dataDF=convertDataFrame(data_path)
        
        # DataFrame ==> 피쳐와 타겟 추출
        self.featuresDF=dataDF[dataDF.columns[:-1]]
        self.targetDF=dataDF[dataDF.columns[-1:]]
        
        # 타겟 Series 정수 인코딩
        result = check_type(self.targetDF[dataDF.columns[-1:]])
        if result is not None: self.targetDF=result.to_frame()
        
        # 데이터셋의 행과 열 수 저장 
        self.n_features=self.featuresDF.shape[1]
        self.n_rows=self.featuresDF.shape[0]
        
    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows
    
    
    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor 반환!!!
    def __getitem__(self, idx):
        featureTS=torch.FloatTensor( self.featuresDF.iloc[idx].values )
        targetTS=torch.FloatTensor( self.targetDF.iloc[idx].values )
        return featureTS, targetTS

In [143]:
### [테스트]
# IRIS 데이터셋 인스턴스 생성
DATA_PATH='../data/iris.csv'
irisFDS= FileDataset(DATA_PATH)

In [144]:
# IRIS 데이터셋 속성
irisFDS.n_features,  irisFDS.n_rows, irisFDS.featuresDF.shape, irisFDS.targetDF.shape

(4, 150, (150, 4), (150, 1))

In [145]:
irisFDS.targetDF.value_counts()

variety
0          50
1          50
2          50
Name: count, dtype: int64

[3] 데이터셋 인스턴스 생성 <hr>

In [146]:
# 피쳐와 타겟 추출
featureDF, targetDF =irisDF[irisDF.columns[:-1]], irisDF[irisDF.columns[-1:]]
print(f'featureDF => {featureDF.shape}, targetDF => {targetDF.shape}')

featureDF => (150, 4), targetDF => (150, 1)


In [147]:
# IRIS 데이터셋 인스턴스 생성
irisDS= CustomDataset(featureDF, targetDF)

In [148]:
# IRIS 데이터셋 속성
irisDS.n_features,  irisDS.n_rows

(4, 150)

In [149]:
# IRIS 데이터셋 메서드
irisDS[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

[4] 데이터로더 인스턴스 생성

In [152]:
## 필요한 것 : Dataset 인스턴스, Batch_size=1[기]
irisDL=DataLoader(irisDS, batch_size=2)

In [154]:
for dataTS, targetTS in irisDL:
    print(dataTS.shape, targetTS.shape)
    print(dataTS, targetTS, sep='\n')
    break

torch.Size([2, 4]) torch.Size([2, 1])
tensor([[5.1000, 3.5000, 1.4000, 0.2000],
        [4.9000, 3.0000, 1.4000, 0.2000]])
tensor([[0.],
        [0.]])
